#### Inc Control, S1
- Look at the changes after 60 min of incubation
- If largely unchanged, then it suggests it will be OK to use the incubation conditions as pseudo-steady-state

In [ ]:
import os
import pandas as pd
import sys
import numpy as np
import scipy.stats as stats
import matplotlib.ticker as plticker

sys.path.append('../scripts')
from plot_helpers import *
from plotting_fxns import *
from utilities import load_dataset

%load_ext autoreload
%autoreload 2

outdir = '../Figures/IncTest'
os.makedirs(outdir, exist_ok = True)

In [ ]:
# The _filtered.csv file already has the spike-ins and rRNA genes removed
res_file = os.path.join(results_dir_inctest, 'gene_quantification','summary_abundance_by_gene_filtered.csv')
df1 = load_dataset(res_file, '../Figures/summary_files/brainInc_mock_passed.csv')
# get the means by experiment
mean_df1 = df1.groupby(['gene', 'condition', 'RNAtype']).mean()
df2 = load_dataset(res_file, '../Figures/summary_files/brainInc_foursu_passed.csv')
# get the means by experiment
mean_df2 = df2.groupby(['gene', 'condition', 'RNAtype']).mean()

In [ ]:
# Compare the 60 min input samples to the 0 min input samples
val_col = 'summed_tpm_recalc'
experiments_mock = [{'condition':'0mock', 'RNAtype':'input'}, {'condition':'60mock', 'RNAtype':'input'}]
experiments_foursu = [{'condition':'0mock', 'RNAtype':'input'}, {'condition':'60foursu', 'RNAtype':'input'}]
limits = {'x':{'left':-2}, 'y':{'bottom':-2}}
loc2 = plticker.MultipleLocator(base=2.0)

mock_cdf = compare_experiments(mean_df1.reset_index(), experiments=experiments_mock, id_col='gene', val_col=val_col, pseudo='min', log=True)
fig = plt.figure(figsize=(dfig, dfig), constrained_layout=True)
ax = fig.add_subplot(111)
ax = plot_scatter(mock_cdf, experiments=['summed_tpm_recalc_x', 'summed_tpm_recalc_y'], id_col='gene',
            xlabel='0 min inc. RNA level\n (log'r'$_{10}$'' TPM)', ylabel='60 min inc. RNA level\n (log'r'$_{10}$'' TPM)', rsquare=True,
            histscatter=True, same_max=True, limits=limits, loc=loc2, ax=ax)
ax.set_aspect('equal')
ax.set_ylabel(ax.get_ylabel(), loc='bottom', ma='center')
plt.savefig('%s.%s' % (os.path.join(outdir, '60vs0mock_histscat'), out_fmt), dpi = out_dpi)

# Compare the 60 min foursu input samples to the 0 min input samples
foursu_cdf = compare_experiments(mean_df2.reset_index(), experiments=experiments_foursu, id_col='gene', val_col=val_col, pseudo='min', log=True)
fig = plt.figure(figsize=(dfig, dfig), constrained_layout=True)
ax = fig.add_subplot(111)
ax = plot_scatter(foursu_cdf, experiments=['summed_tpm_recalc_x', 'summed_tpm_recalc_y'], id_col='gene',
            xlabel='0 min inc. RNA level\n (log'r'$_{10}$'' TPM)', ylabel='60 min inc. +4sU RNA\nlevel (log'r'$_{10}$'' TPM)', rsquare=True,
            histscatter=True, limits=limits, same_max=True, loc=loc2, ax=ax)
ax.set_aspect('equal')
ax.set_ylabel(ax.get_ylabel(), loc='bottom', ma='center')
plt.savefig('%s.%s' % (os.path.join(outdir, '60vs0foursu_histscat'), out_fmt), dpi = out_dpi)

In [ ]:
# Calculate the k3/k2 (deg/proc) ratios using the P/M ratio (as discussed in Furlan et al., 2020, Fig. 1B)
inspectminus_dir = '/Users/mk/Desktop/Davislab/3.10_brain_stability/brain_results/brain_inctest2_inspect_minus'

mature_df = pd.read_csv(os.path.join(inspectminus_dir, 'mature.csv'), index_col=0)
premature_df = pd.read_csv(os.path.join(inspectminus_dir, 'premature.csv'), index_col=0)
PM_df = premature_df/mature_df
PM_df_log = PM_df.apply(np.log10).replace([np.inf, -np.inf], np.nan).dropna()
passed_genes_mock = pd.read_csv('../Figures/summary_files/brainInc_mock_passed.csv', header=None)[0]
passed_genes_foursu = pd.read_csv('../Figures/summary_files/brainInc_foursu_passed.csv', header=None)[0]
PM_df1 = PM_df[PM_df.index.isin(passed_genes_mock)].copy()
PM_df2 = PM_df[PM_df.index.isin(passed_genes_foursu)].copy()
PM_df_log1 = PM_df_log[PM_df_log.index.isin(passed_genes_mock)].copy()
PM_df_log2 = PM_df_log[PM_df_log.index.isin(passed_genes_foursu)].copy()

In [ ]:
# Compare k3/k2 ratios between samples
val_col = 'summed_tpm_recalc'
experiments_mock = [{'condition':'0mock', 'RNAtype':'input'}, {'condition':'60mock', 'RNAtype':'input'}]
experiments_foursu = [{'condition':'0mock', 'RNAtype':'input'}, {'condition':'60foursu', 'RNAtype':'input'}]
# limits = {'x':{'left':-2}, 'y':{'bottom':-2}}
limits = None
loc = None

fig = plt.figure(figsize=(dfig, dfig), constrained_layout=True)
ax = fig.add_subplot(111)
ax = plot_scatter(PM_df_log1.reset_index(), experiments=['0mock', '60mock'], id_col='index',
            xlabel='0 min inc.\ndeg / proc rate', ylabel='60 min inc.\ndeg / proc rate', rsquare=True,
            histscatter=True, same_max=True, limits=limits, loc=loc2, ax=ax)
ax.set_aspect('equal')
# ax.set_ylabel(ax.get_ylabel(), loc='bottom', ma='center')
plt.savefig('%s.%s' % (os.path.join(outdir, '60vs0mock_histscat_PM'), out_fmt), dpi = out_dpi)

fig = plt.figure(figsize=(dfig, dfig), constrained_layout=True)
ax = fig.add_subplot(111)
ax = plot_scatter(PM_df_log1.reset_index(), experiments=['0mock', '60foursu'], id_col='index',
            xlabel='0 min inc.\ndeg / proc rate', 
            ylabel='60 min inc. +4sU\ndeg / proc rate', rsquare=True,
            histscatter=True, same_max=True, limits=limits, loc=loc2, ax=ax)
ax.set_aspect('equal')
# ax.set_ylabel(ax.get_ylabel(), loc='bottom', ma='center')
plt.savefig('%s.%s' % (os.path.join(outdir, '60vs0foursu_histscat_PM'), out_fmt), dpi = out_dpi)